In [ ]:
%matplotlib notebook
#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os, os.path
import glob
import h5py
import pickle
import numpy as np
import pynbody
from numpy.polynomial import Polynomial
from mpl_toolkits import mplot3d
from scipy import ndimage, signal, interpolate
from astropy.coordinates import SkyCoord,Galactic,ICRS,Galactocentric
import astropy.coordinates as coord
from galpy.potential import NFWPotential,DoubleExponentialDiskPotential,MN3ExponentialDiskPotential,HernquistPotential,ChandrasekharDynamicalFrictionForce
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014,MiyamotoNagaiPotential
from galpy.util import bovy_conversion, save_pickles, bovy_coords, bovy_plot
#import seaborn as sns
from matplotlib import animation
from scipy.stats import gaussian_kde
import astropy.units as u
from scipy.interpolate import interpn
from galpy import potential
from galpy.potential import InterpSnapshotRZPotential
import pickle
from galpy.potential import vcirc
from galpy.util import bovy_conversion, bovy_plot, save_pickles

#from matplotlib import cm, pyplot
from galpy.potential import evaluatezforces

coord.galactocentric_frame_defaults.set('v4.0') 

In [ ]:
ro=8.
vo=220.

    
def read_snapshot(fname='test.hdf5',read_axes='Coordinates',ptype=1):
    data = h5py.File(fname, 'r')
    group = data['PartType{}'.format(int(ptype))]
    
    try:
        dd=group[read_axes][()]
        try:
            if np.size(dd,1) ==3:
                return (dd[:,0],dd[:,1],dd[:,2])
    
        except IndexError:
            return (dd)
    
    except KeyError:
        print ("Allowed axes names %s"%group.keys())
        

def Gizmo_to_pynbody(fn="test.hdf5",ptypes=[1,2,3,4,5],eps_list=[0.1,1.0,1.0,1.0,1.0]):
    '''
    We are going to merge all ptypes to dm
    '''
    x=[]
    y=[]
    z=[]
    vx=[]
    vy=[]
    vz=[]
    m=[] 
    eps=[]
    for pt in ptypes:
        _x,_y,_z = read_snapshot(fname=fn,read_axes='Coordinates',ptype=pt)
        _vx,_vy,_vz = read_snapshot(fname=fn,read_axes='Velocities',ptype=pt)
        _m=read_snapshot(fname=fn,read_axes='Masses',ptype=pt)
        
        _x = _x.tolist()
        _y = _y.tolist()
        _z = _z.tolist()
        _vx = _vx.tolist()
        _vy = _vy.tolist()
        _vz = _vz.tolist()
        _m = _m.tolist()
        
        x.append(_x)
        y.append(_y)
        z.append(_z)
        vx.append(_vx)
        vy.append(_vy)
        vz.append(_vz)
        m.append(_m)
        
        eps.append(np.ones(len(_x),dtype=np.float64).tolist())
        
    x = np.array([item for sublist in x for item in sublist])
    y = np.array([item for sublist in y for item in sublist])
    z = np.array([item for sublist in z for item in sublist])
    vx = np.array([item for sublist in vx for item in sublist])
    vy = np.array([item for sublist in vy for item in sublist])
    vz = np.array([item for sublist in vz for item in sublist])
    m = np.array([item for sublist in m for item in sublist])
    eps = np.array([item for sublist in eps for item in sublist])

    pos1= np.c_[(x,y,z)]
    vel1= np.c_[(vx,vy,vz)]
    
    vo, ro= 220., 8.
    ##Modify below
    f = pynbody.snapshot.new(dm=len(m))
    f['pos'] =pynbody.array.SimArray(pos1/ro)#,"kpc")
    f['vel'] =pynbody.array.SimArray(vel1/vo)#,'km s**-1')
    f['mass'] =pynbody.array.SimArray(m*1e10/bovy_conversion.mass_in_msol(vo,ro))#,'Msol')
    f['eps'] = pynbody.array.SimArray(eps/ro)#,'kpc')
    f.physical_units()
    
    return (f)
  

In [ ]:
#convert Gizmo to pynbody and interpolate

ff = Gizmo_to_pynbody(fn="MW_GB_Mres1e6Msun_GD-1_halovel_flippedInitialPosition.hdf5",ptypes=[1])

from galpy.util.bovy_conversion import _G
g= pynbody.array.SimArray(_G/1000.)
g.units= 'kpc Msol**-1 km**2 s**-2 G**-1'
ff._arrays['mass']= ff._arrays['mass']*g

spi= InterpSnapshotRZPotential(ff,rgrid=(np.log(0.01),np.log(100./ro),101),logR=True,zgrid=(0.,50./ro,101),interpPot=True,\
                               zsym=True,numcores=6)


print (vcirc(spi,1))
delattr(spi,'_s')
delattr(spi,'_origPot')

save_pickles("Potential_MWHalo_Mres1e6.pkl",spi)

In [ ]:
#load the saved pickle file

from galpy.potential import vcirc

with open("Potential_MWHalo_Mres1e6.pkl",'rb') as savefile:
    newspi= pickle.load(savefile,encoding='latin1')

print (vcirc(newspi, 1),vcirc(spi,1))
